# AB test 

A/B testing is a research method that allows you to find out people's reaction to any changes. The study shows which of the two versions of the product or offer is better and gives greater effect.

In [1]:
import random

import numpy as np

from hypex.dataset import Dataset, InfoRole, TreatmentRole, TargetRole
from hypex.experiments.ab import ABTest

## Creation of a new test dataset with synthetic data.
It is important to mark the data fields by assigning the appropriate roles:

* FeatureRole: a role for columns that contain features or predictor variables. Our split will be based on them. Applied by default if the role is not specified for the column.
* TreatmentRole: a role for columns that show the treatment or intervention.
* TargetRole: a role for columns that show the target or outcome variable.
* InfoRole: a role for columns that contain information about the data, such as user IDs.

In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole(), 
        "gender": TargetRole()
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
data["treat"] = [random.choice([0, 1, 2]) for _ in range(len(data))]
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      0       512.5   462.222222  26.0    NaN   
2           2             7      0       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      0       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      0       538.5   450.444444  42.0      M   
9996     9996             0      2       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      2       495.0   523.222222  67.0      F   
9999     9999             7      0       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [4]:
data.roles

{'user_id': Info(<class 'int'>),
 'treat': Treatment(<class 'int'>),
 'pre_spends': Target(<class 'float'>),
 'post_spends': Target(<class 'float'>),
 'gender': Target(<class 'str'>),
 'signup_month': Feature(<class 'int'>),
 'age': Feature(<class 'float'>),
 'industry': Feature(<class 'str'>)}

## Simple AB Test 

Simple pipline contains group sizes, group differences and TTest estimation. 

In [5]:
test = ABTest()
result = test.execute(data)

In [6]:
result.resume

       feature group TTest pass  TTest p-value  control mean   test mean  \
0   pre_spends     1     NOT OK       0.147842    486.785516  487.455726   
1   pre_spends     2     NOT OK       0.599749    451.159904  452.423390   
2  post_spends     1     NOT OK       0.192402    486.785516  487.026582   
3  post_spends     2     NOT OK       0.076589    451.159904  452.872356   

   difference  difference %  
0    0.670210      0.137681  
1    1.263486      0.280053  
2    0.241065      0.049522  
3    1.712452      0.379566  

In [7]:
result.sizes 

   control size  test size  control size %  test size % group
1          3245       3388              48           51     1
2          3245       3367              49           50     2

In [8]:
result.multitest

"There was less than three groups or multitest method wasn't provided"

## Additional tests in AB Test 

It is possible to add u-test and chi2-test in pipline. 

In [9]:
test = ABTest(additional_tests=['t-test', 'u-test', 'chi2-test'])
result = test.execute(data)

In [10]:
result.resume

       feature group TTest pass  TTest p-value UTest pass  UTest p-value  \
0   pre_spends     1     NOT OK       0.147842     NOT OK       0.391934   
1   pre_spends     2     NOT OK       0.599749     NOT OK       0.825443   
2  post_spends     1     NOT OK       0.192402     NOT OK       0.178752   
3  post_spends     2     NOT OK       0.076589         OK       0.035613   

  Chi2Test pass  Chi2Test p-value  control mean   test mean  difference  \
0           NaN               NaN    486.785516  487.455726    0.670210   
1           NaN               NaN    451.159904  452.423390    1.263486   
2           NaN               NaN    486.785516  487.026582    0.241065   
3           NaN               NaN    451.159904  452.872356    1.712452   

   difference %  
0      0.137681  
1      0.280053  
2      0.049522  
3      0.379566  

In [11]:
result.multitest

"There was less than three groups or multitest method wasn't provided"

In [12]:
result.sizes

   control size  test size  control size %  test size % group
1          3245       3388              48           51     1
2          3245       3367              49           50     2

## ABn Test 

Finally, we may estimate multiple ab test with different methods.

In [13]:
test = ABTest(multitest_method="bonferroni")
result = test.execute(data)

In [14]:
result.resume

       feature group TTest pass  TTest p-value  control mean   test mean  \
0   pre_spends     1     NOT OK       0.147842    486.785516  487.455726   
1   pre_spends     2     NOT OK       0.599749    451.159904  452.423390   
2  post_spends     1     NOT OK       0.192402    486.785516  487.026582   
3  post_spends     2     NOT OK       0.076589    451.159904  452.872356   

   difference  difference %  
0    0.670210      0.137681  
1    1.263486      0.280053  
2    0.241065      0.049522  
3    1.712452      0.379566  

In [15]:
result.sizes

   control size  test size  control size %  test size % group
1          3245       3388              48           51     1
2          3245       3367              49           50     2

In [16]:
result.multitest

   correction        field  new p-value  old p-value  rejected   test group
0    0.250000   pre_spends     0.591369     0.147842     False  TTest     1
1    0.599749  post_spends     1.000000     0.599749     False  TTest     1
2    0.250000   pre_spends     0.769606     0.192402     False  TTest     2
3    0.250000  post_spends     0.306357     0.076589     False  TTest     2